Question: How does the population per county affect the rate of accidents.

In [3]:
import os
# from os import path

# Module for reading CSV files
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from census import Census

from JacksConfig import api_key
c=Census(api_key, year=2020)

All_accident = 'Resources/Cleaned2020Data.csv'
All_accident_data = pd.read_csv(All_accident)
All_accident_data

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Sunrise_Sunset,year
0,232630,A-232631,2,2020-12-30 22:58:00,2021-08-28 15:14:32,39.775174,-75.480441,39.790290,-75.464704,1.338,...,2020-12-30 22:51:00,44.0,37.0,55.0,10.0,14.0,0.0,Cloudy,Night,2020
1,246924,A-246925,2,2020-10-13 19:19:34.000000000,2021-07-19 08:00:00.000000000,44.725203,-122.185579,44.734005,-122.166744,1.107,...,2020-10-13 18:56:00,60.0,60.0,64.0,10.0,7.0,0.0,Cloudy,Night,2020
2,247119,A-247120,2,2020-05-23 09:23:48.000000000,2021-06-01 10:45:00.000000000,44.875127,-121.428011,44.846357,-121.397251,2.494,...,2020-05-23 09:53:00,59.0,59.0,48.0,10.0,18.0,0.0,Fair,Day,2020
3,250560,A-250561,2,2020-10-13 19:19:34,2021-10-01 08:00:00,44.704907,-122.231897,44.697982,-122.217820,0.841,...,2020-10-13 18:56:00,60.0,60.0,64.0,10.0,7.0,0.0,Cloudy,Night,2020
4,279132,A-279133,2,2020-12-30 22:58:00,2021-03-31 20:49:43,39.783118,-75.473129,39.791412,-75.462887,0.790,...,2020-12-30 22:51:00,44.0,37.0,55.0,10.0,14.0,0.0,Cloudy,Night,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625859,2362657,A-2362658,2,2020-01-06 20:59:00,2020-01-06 21:32:38,34.265778,-118.454289,34.265778,-118.454289,0.000,...,2020-01-06 19:50:00,59.0,59.0,22.0,10.0,9.0,NaN,Fair,Night,2020
625860,2362658,A-2362659,2,2020-01-06 21:14:00,2020-01-06 23:16:38,35.736667,-119.742500,35.736667,-119.742500,0.000,...,2020-01-06 20:54:00,43.0,43.0,86.0,5.0,3.0,0.0,Fair,Night,2020
625861,2362659,A-2362660,2,2020-01-06 21:19:00,2020-01-06 22:19:37,34.075263,-118.281157,34.075263,-118.281157,0.000,...,2020-01-06 20:52:00,64.0,64.0,24.0,10.0,5.0,0.0,Fair,Night,2020
625862,2372960,A-2372961,2,2020-01-01 00:08:02,2020-01-01 00:37:03,42.315690,-83.085920,42.312500,-83.094120,0.473,...,2020-01-01 00:53:00,28.0,19.0,92.0,7.0,9.0,0.0,Cloudy,Night,2020


In [ ]:
list(All_accident_data.columns)

In [4]:
#git the the data I need for my question
Per_countyDF = All_accident_data.groupby(['State','County']).count()
Per_countyDF = Per_countyDF.drop(columns=['ID','Severity', 'Start_Time', 'End_Time','Start_Lat',
 'Start_Lng','End_Lat','End_Lng','Distance(mi)','Number', 'Street','Side','City',
 'Zipcode','Country','Weather_Timestamp','Temperature(F)','Wind_Chill(F)','Humidity(%)',
 'Visibility(mi)','Wind_Speed(mph)','Precipitation(in)', 'Weather_Condition','Sunrise_Sunset',
 'year'])
Per_countyDF["# of Accidents"] = Per_countyDF["Unnamed: 0"]
Per_countyDF = Per_countyDF.drop(columns=["Unnamed: 0"])
Per_countyDF = Per_countyDF.reset_index()
Per_countyDF

,State,County,# of Accidents
0,AL,Autauga,67
1,AL,Baldwin,195
2,AL,Barbour,7
3,AL,Bibb,6
4,AL,Blount,142
...,...,...,...
2127,WV,Wood,32
2128,WY,Carbon,1
2129,WY,Fremont,2
2130,WY,Sweetwater,5


In [5]:
Per_countyDF
Per_countyDF.dtypes

State             object
County            object
# of Accidents     int64
dtype: object

In [6]:
#Use for abbreviating state names below
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [7]:
#Get census data per county 
# Create Census object with Census API key from the selected year
c = Census(api_key, year=2020)
# Run Census Search to retrieve income data by county
census_data = c.acs5.get(("NAME", "B01002_001E"),{'for':'county:*'})
# Convert to DataFrame
census_df = pd.DataFrame(census_data)
census_df.head()
# Rename columns
census_df = census_df.rename(columns={"B01002_001E": "Median Age",
                                     "NAME": "County",
                                     "State": "State Code",
                                     "County": "County Code"})
census_df.head()
# Split “County” column into separate “County” and “State” columns
census_df[["County", "State"]] = census_df["County"].str.split("County, ", expand=True)
# Combine State and County codes into new column for county identification
census_df["County FIPS"] = census_df["State"]+" " + census_df["County"]
# Drop State and County code columns
census_df.drop(columns=["state", "county", "County FIPS"], inplace=True)
# Convert State names to state abbreviations
census_df.replace({"State": us_state_to_abbrev}, inplace=True)
# census_pd.replace({“State”: states_abbreviation}, inplace=True)
#Remove trailing spaces from the county column 
census_df['County']= census_df['County'].str.strip()
census_df

,County,Median Age,State
0,Autauga,38.6,AL
1,Baldwin,43.2,AL
2,Barbour,40.1,AL
3,Bibb,39.9,AL
4,Blount,41.0,AL
...,...,...,...
3216,Renville,44.0,MN
3217,Roseau,41.6,MN
3218,Sherburne,36.1,MN
3219,Steele,39.2,MN


In [8]:
#Combine the census DF and the accident DF
Accident_AgeDF = pd.merge(Per_countyDF, census_df, how="left", on=["State", "County"])
Accident_AgeDF

,State,County,# of Accidents,Median Age
0,AL,Autauga,67,38.6
1,AL,Baldwin,195,43.2
2,AL,Barbour,7,40.1
3,AL,Bibb,6,39.9
4,AL,Blount,142,41.0
...,...,...,...,...
2127,WV,Wood,32,43.8
2128,WY,Carbon,1,39.2
2129,WY,Fremont,2,38.5
2130,WY,Sweetwater,5,35.7
